In [1]:
#Importar librerías
#============================================================================
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Input, Dense
from keras.utils import to_categorical
from keras.models import Model

import warnings
warnings.filterwarnings('ignore')

2023-06-07 21:32:59.642782: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#importar pyspark
#============================================================================
import pyspark
from pyspark import SparkContext
sc =SparkContext()


23/06/07 21:33:11 WARN Utils: Your hostname, AMAC02XJGZUJG5H-2.local resolves to a loopback address: 127.0.0.1; using 192.168.1.135 instead (on interface en0)
23/06/07 21:33:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/07 21:33:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#inicializar SQLContext
#============================================================================
from pyspark import SQLContext
sqlContext = SQLContext(sc)

1. Lee los datos con PySpark

In [4]:
df = sqlContext.read.csv("2008.csv", sep = ",", inferSchema = True, header = True)

2. Explorar los Datos:

    Explora los datos para familiarizarte con ellos. Aquí hay algunas cosas que podrías hacer:
    * Mostrar las primeras filas del objeto con los datos.
    * Ver la cantidad de filas y columnas del objeto con los datos.
    * Mostrar el esquema del objeto con los datos. Qué tipo de objeto es?


In [5]:
#Mostrar datos
#============================================================================
df.show(5)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2008|    1|         3|        4|   1343|      1325|   1451|      1435|           WN|      588

23/06/07 21:33:18 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
# Obtener la cantidad de filas
#============================================================================
num_filas = df.count()
print("----------------------------")
print("Número de filas:", num_filas)

# Obtener la cantidad de columnas
#============================================================================
num_columnas = len(df.columns)
print("Número de columnas:", num_columnas)
print("----------------------------")


----------------------------
Número de filas: 2389217
Número de columnas: 29
----------------------------


In [7]:
#Mostrar esquema - revisar tipo de datos
#============================================================================
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

In [8]:
#Revisar el tipo de objeto
#============================================================================
type(df)

pyspark.sql.dataframe.DataFrame

3. Limpieza de Datos

    Asegúrate de que los datos estén limpios. Puedes necesitar hacer cosas como:
    * Eliminar filas con datos faltantes.
    * Convertir columnas a los tipos de datos correctos.


In [9]:
from pyspark.sql.functions import col

#Obtener los nombres de las columas
#============================================================================
columnas = df.columns

# Revisar las cantidad de filas con NA para cada columna
#============================================================================
for columna in columnas:
    df_na = df.filter(col(columna).isNull()).count()
    if df_na > 0:
        print(f"Filas con valores nulos en la columna '{columna}': {df_na}")
    

Filas con valores nulos en la columna 'TailNum': 42452


Filas con valores nulos en la columna 'CancellationCode': 2324775


In [10]:
'''
Existen 2 variables que tienen una cantidad importante de NA, estos corresponden a códigos y por lo tanto no son de utilidad
en el análisis, por lo que se eliminarán
'''

#Eliminar columnas TailNum y CancellationCode
#============================================================================
df_clean = df.drop("TailNum","CancellationCode")
df_clean.columns

['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DepTime',
 'CRSDepTime',
 'ArrTime',
 'CRSArrTime',
 'UniqueCarrier',
 'FlightNum',
 'ActualElapsedTime',
 'CRSElapsedTime',
 'AirTime',
 'ArrDelay',
 'DepDelay',
 'Origin',
 'Dest',
 'Distance',
 'TaxiIn',
 'TaxiOut',
 'Cancelled',
 'Diverted',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay']

In [11]:
# Existen datos NA, pero ingresados como str, por lo que se revisarán y eliminarán.

#Obtener los nombres de las columas
#============================================================================
columnas = df_clean.columns

# Revisar las cantidad de filas con "NA" para cada columna
#============================================================================
for columna in columnas:
    df_na = df_clean.filter(col(columna).like("%NA%")).count()
    if df_na > 0:
        print(f"Filas con valores nulos en la columna '{columna}': {df_na}")

Filas con valores nulos en la columna 'DepTime': 64442
Filas con valores nulos en la columna 'ArrTime': 70096


Filas con valores nulos en la columna 'ActualElapsedTime': 70096
Filas con valores nulos en la columna 'CRSElapsedTime': 407
Filas con valores nulos en la columna 'AirTime': 70096


Filas con valores nulos en la columna 'ArrDelay': 70096


Filas con valores nulos en la columna 'DepDelay': 64442


Filas con valores nulos en la columna 'Origin': 40481


...

Filas con valores nulos en la columna 'TaxiOut': 64442


Filas con valores nulos en la columna 'CarrierDelay': 1804634


Filas con valores nulos en la columna 'WeatherDelay': 1804634


Filas con valores nulos en la columna 'NASDelay': 1804634


Filas con valores nulos en la columna 'SecurityDelay': 1804634
Filas con valores nulos en la columna 'LateAircraftDelay': 1804634


In [12]:
'''
Dado lo solicitado en los análisis, las variables en las que se eliminarán los datos perdidos son: 
1. 'ActualElapsedTime' ya que se pide obtener cuál es el vuelo más largo
2. 'Dest', puesto se requiere saber cuál es el destino más popular.
'''

# Construir lista con las columnas a considerar
#============================================================================
columnas = ['ActualElapsedTime', 'Dest']

# Eliminar las filas con "NA" en las columnas de tipo string
#============================================================================
for columna in columnas:
    if df_clean.schema[columna].dataType == 'string':
        df_clean = df_clean.filter(~col(columna).like("%NA%"))

# Mostrar el DataFrame resultante sin las filas que contenían "NA"
#============================================================================
df_clean.show(5)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+--------+------------+------------+--------+-------------+-----------------+
|2008|    1|         3|        4|   1343|      1325|   1451|      1435|           WN|      588|               68|            70|     55|      16|      18|   HOU| LIT|   

In [13]:
# Convertir tipo de datos
'''
Solamente se casteará la var 'ActualElapsedTime' ya que es variable de interés.
Existen otras columnas que se deberían modificar; sin embargo, por tener "NA", no se puede.
'''
#============================================================================

from pyspark.sql import types

df_clean = df_clean.withColumn('ActualElapsedTime', df_clean['ActualElapsedTime'].cast(types.IntegerType()))

df_clean.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- ActualElapsedTime: integer (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- CarrierDelay: string (nullable = true)
 |-- WeatherDelay: string (nullable = true)
 |-- N

4. Análisis de Datos

    Realiza un análisis básico de los datos.
    * ¿Cuál es el vuelo más largo? Que unidad de tiempo es más apropiada? De qué depende? Seleccione una.
    * ¿Cuál es la aerolínea con más vuelos?
    * ¿Cuál es el destino más popular?
    * ¿Qué día de la semana tiene más vuelos?


In [14]:
######################### Determinar vuelo más largo #########################

#Calcular la mayor duración de un vuelo
#============================================================================
from pyspark.sql import functions as F

max_vuelo = df_clean.select(F.max("ActualElapsedTime")).first()[0]
print(f"El tiempo máximo de vuelo es: {max_vuelo} minutos")


El tiempo máximo de vuelo es: 905 minutos


In [15]:
'''
El tiempo máximo es 905 minutos, por lo que la mejor unidad es en horas. Esto va a depender de cuánta duración tengan los
vuelos que se están analizando. En casos en que el máximo no sea mucho mayor que 60 minutos es preferible dejarlo en minutos,
de lo contrario es mejor las horas, para poder tener una mejor idea del tiempo transcurrido.
'''

# Traspasar de minutos a hora la variable "ActualElapsedTime"
#============================================================================
df_with_hours = df_clean.withColumn("ActualElapsedTime_hours", F.col("ActualElapsedTime") / 60)

#Calcular la mayor duración de un vuelo
#============================================================================
max_vuelo_hrs = df_with_hours.select(F.max("ActualElapsedTime_hours")).first()[0]
print(f"El tiempo máximo de vuelo es: {max_vuelo_hrs:.2f} horas")

El tiempo máximo de vuelo es: 15.08 horas


In [16]:
#Obtener el listado de vuelos que tienen la duración máxima
#============================================================================
df_max_vuelo = df_with_hours.filter(F.col("ActualElapsedTime") == max_vuelo)
df_max_vuelo = df_max_vuelo.select("FlightNum", "UniqueCarrier","Origin", "Dest", "ActualElapsedTime_hours")

df_max_vuelo.show()

+---------+-------------+------+----+-----------------------+
|FlightNum|UniqueCarrier|Origin|Dest|ActualElapsedTime_hours|
+---------+-------------+------+----+-----------------------+
|       22|           HA|   HNL| SEA|     15.083333333333334|
+---------+-------------+------+----+-----------------------+



In [17]:
######################### Determinar la aerolinea con mayor cantidad de vuelos #########################

#Genear df con la cuenta de viajes por aerolinea
#============================================================================
df_vuelos_aerolinas = df_with_hours.groupBy("UniqueCarrier").count()

#Obtener el la cantidad máxima de viajes
#============================================================================
max_aerolinea = df_vuelos_aerolinas.select(F.max("count")).first()[0]

#Obtener la aerolinea que tiene la mayor cantidad de vuelos
#============================================================================
df_max_aerolinea = df_vuelos_aerolinas.filter(F.col("count") == max_aerolinea)
aerolinea = df_max_aerolinea.collect()[0][0]
vuelos = df_max_aerolinea.collect()[0][1]

print(f"La aerolínea con más vuelos es: {aerolinea}, con {vuelos} viajes")

La aerolínea con más vuelos es: WN, con 398966 viajes


In [18]:
######################### Determinar cuál es el destino #########################

#Genear df con la cuenta de destinos
#============================================================================
df_vuelos_destino = df_with_hours.groupBy("Dest").count()

#Obtener el la cantidad máxima de viajes
#============================================================================
max_destino = df_vuelos_destino.select(F.max("count")).first()[0]

#Obtener el destino que tiene la mayor cantidad de vuelos
#============================================================================
df_max_destino = df_vuelos_destino.filter(F.col("count") == max_destino)

destino = df_max_destino.collect()[0][0]
vuelos = df_max_destino.collect()[0][1]

print(f"El destino con más vuelos es: {destino}, con {vuelos} viajes")

El destino con más vuelos es: ATL, con 136950 viajes


In [19]:
######################### Día de la semana con más vuelos #########################

#Genear df con la cuenta de viajes por dia
#============================================================================
df_vuelos_dia = df_with_hours.groupBy("DayOfWeek").count()

#Obtener el la cantidad máxima de viajes
#============================================================================
max_dia = df_vuelos_dia.select(F.max("count")).first()[0]

#Obtener el dia que tiene la mayor cantidad de vuelos
#============================================================================
df_max_dia= df_vuelos_dia.filter(F.col("count") == max_dia)
df_max_dia.show()

+---------+------+
|DayOfWeek| count|
+---------+------+
|        3|365560|
+---------+------+



In [20]:
#Reemplazar número por nombre del día
#============================================================================
df_max_dia = df_max_dia.withColumn("DayOfWeek", F.when(F.col("DayOfWeek") == 3, "Miércoles"))

#Imprimir resultado análisis
#============================================================================
dia = df_max_dia.collect()[0][0]
vuelos = df_max_dia.collect()[0][1]

print(f"El día con más vuelos es: {dia}, con {vuelos} viajes")

El día con más vuelos es: Miércoles, con 365560 viajes


In [21]:
#cerrar sesión
sc.stop()